In [8]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集
我们可以调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据

In [9]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    #TensorDataset将两个tensor放在一起,按照tensor的第一维度
    #单个星号代表这个位置接收任意多个非关键字参数，在函数的*位置上将其转化成元组
    #而双星号代表这个位置接收任意多个关键字参数，在**位置上将其转化成字典
    # *data_arrays是元祖 tuple
    #参考：https://blog.csdn.net/weixin_46805040/article/details/122747290
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
batch_size = 10
data_iter = load_array((features,labels),batch_size)

In [10]:
next(iter(data_iter))

[tensor([[ 0.3568,  1.2684],
         [-0.3454, -1.4369],
         [ 2.0237, -0.1875],
         [-0.0556,  1.4198],
         [-1.0721, -0.4287],
         [-0.2157, -0.3968],
         [ 0.2541,  0.3825],
         [ 1.4712,  0.1552],
         [ 0.1856,  1.1697],
         [-0.3511,  1.4287]]), tensor([[ 0.5948],
         [ 8.3861],
         [ 8.8932],
         [-0.7287],
         [ 3.5041],
         [ 5.1093],
         [ 3.4141],
         [ 6.6151],
         [ 0.5844],
         [-1.3622]])]

## 构建网络

In [11]:
from torch import nn
#bias默认为True
net = nn.Sequential(nn.Linear(2,1,bias=True))

## 网络参数初始化
在使用net之前，我们需要初始化模型参数。 如在线性回归模型中的权重和偏置。 深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

In [12]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 构建损失函数

In [13]:
#默认用均方误差
loss = nn.MSELoss()

## 定义优化算法
小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [14]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

In [15]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000285
epoch 2, loss 0.000095
epoch 3, loss 0.000095


In [16]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0006, -0.0009])
b的估计误差： tensor([0.0003])
